# 氏名漢字カナ突合モデル

漢字カナ突合モデルはJava17で記述されている。Mavenを用いて漢字カナ突合モデルをビルドする。
仕様等については[README](./README.md)を参照のこと。


## 必要なモジュール

- JDK 17
- Maven 3




## 事前準備
- バージョン
pom.xmlのバージョン番号は、../version.txtに記載

- ファイル
[dict](../dict)で作成した，辞書ファイル及び，[train](../train)で作成したAIモデルのファイルを，Javaのリソースとして使用するために，src/main/resources以下にファイルをコピーする。なお，事前に作成した辞書及び，事前学習済みのモデルを，[resources][https://kktg.digital.go.jp/support/resources/]で配布している。これらのファイルをあらかじめ，../dictおよび，../train/model, ../train/model_r以下にコピーしたのちに，下記スクリプトを実行してもよい。

In [1]:
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        version=l.rstrip()
        break

os.environ["VER"]=version

In [2]:
!echo $VER

1.8o


# 漢字とカナのマッチング

漢字姓名とカナ姓名が入力されたファイルをCSVもしくはTSV形式で用意する。漢字姓名とカナ姓名以外のフィールドが含まれていても良い。
なお，VMパラメタとして`-Xmx8196M`を指定し，8Gのメモリを確保すること。
### 実行クラス
jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain
### パラメタ
- infile
入力ファイル。行に漢字姓名とカナ姓名を含むファイルで，TSVもしくはCSV形式で作成しること。また，ヘッダがあっても良くヘッダがある場合には`has_header`パラメタを指定する。また，漢字姓名とカナ姓名の列を`kanji_idx`と`kana_idx`パラメタで指定する
- okfile
漢字姓名とカナ姓名が一致と判断されたレコードが保存されるファイル
- ngfile
漢字姓名とカナ姓名が不一致と判断されたレコードが保存されるファイル
- logfile
入力ファイルの各行を実行する際のログファイル
- kanji_idx
infileのファイルの漢字姓名の列番号，0から始まる
- kana_idx
infileのファイルのカナ姓名の列番号，0から始まる
- sep
infileの各列の区切り文字を指定する。csvもしくはtsvを指定する。デフォルトcsv
- thread_num
infileに大量の行が含まれる場合には，本プログラムをスレッドで多重化して計算することができる。`thread_num`に1を指定すると（デフォルト），シングルスレッドで実行され，それ以上を値を指定するとその数だけスレッドで計算される。スレッドでケインさんした場合には，`okfile`と`ngfile`，`logfile`の末尾にスレッド番号が付与されたファイルが作成される
- has_header
infileにヘッダ業がある場合には`true`を指定する。デフォルトは`false`であり，infileにヘッダ行は含まれない
- strategy
漢字姓名とカナ姓名の突合モデルをどれを使うかを指定する。BASIC, ONLY_AI, ONLY_DICT, AI, ENSEMBLEから選択し，セットする。    

  |モデル|内容|
  |-----|---|
  |ENSEMBLE||
  |BASIC|姓名辞書，異体字辞書との突合のみ|
  |ONLY_AI|AIモデルとの突合のみ|
  |ONLY_DICT|姓名辞書及び単漢字辞書，異体字辞書との突合のみ|
  |AI|BASIC + ONLY_AI|
  

In [3]:
# 出力結果がoutput.txtに出力される
!java -Xmx8196M -Dlog4j.configurationFile=log4j2.xml -classpath lib/kanjikana_core-$VER-jar-with-dependencies.jar jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain --infile input.txt --outfile outfile.txt --kanji_idx 1 --kana_idx 2 --strategy ENSEMBLE --has_header true --thread_num 1 --sep csv


17:12:50.186 [main] DEBUG ai.djl.repository.zoo.DefaultModelZoo -- Scanning models in repo: class ai.djl.repository.SimpleRepository, /tmp/model_14530032122800154140.tmp
17:12:50.192 [main] DEBUG ai.djl.engine.Engine -- Registering EngineProvider: PyTorch
17:12:50.193 [main] DEBUG ai.djl.engine.Engine -- Found default engine: PyTorch
17:12:50.920 [main] INFO ai.djl.util.Ec2Utils -- DJL will collect telemetry to help us better understand our users’ needs, diagnose issues, and deliver additional features. If you would like to learn more or opt-out please go to: https://docs.djl.ai/docs/telemetry.html for more information.
17:12:50.922 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Loading model with Criteria:
	Application: UNDEFINED
	Input: class ai.djl.ndarray.NDList
	Output: class ai.djl.ndarray.NDList
	Engine: PyTorch
	ModelZoo: ai.djl.localmodelzoo
	No translator supplied

17:12:50.923 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Searching model in specified model zoo: ai.djl.local

## 結果を表示する

In [11]:
!cat outfile.txt

null,start_date,end_date
1,山田　太郎,ヤマダ　ハナコ,ng,true,90,{true;山田;ヤマダ;WordEngine;DictReliableNormalized;0:/canna/freewnn/ipadic/kakasi/mozc/;null}{true;太;ハナ;NandokuEngine;;0:/mj/;null}{true;郎;コ;NandokuEngine;;0:/mozc/;null},2025-10-23 17:12:49.005 +0900,2025-10-23 17:12:49.038 +0900
2,山田　太郎,ヤマダタロウ,ok,true,90,{true;山;ヤマ;NandokuEngine;;0:/canna/freewnn/ipadic/kakasi/mj/mozc/skk/;null}{true;太;ロウ;NandokuEngine;;0:/canna/freewnn/ipadic/kakasi/mj/mozc/;null},2025-10-23 17:12:49.038 +0900,2025-10-23 17:12:49.040 +0900
3,山田　太郎,サンダ　フトロウ,ok,true,90,{true;山;サン;NandokuEngine;;0:/canna/freewnn/ipadic/kakasi/mj/mozc/;null}{true;田;ダ;NandokuEngine;;0:/canna/freewnn/kakasi/mozc/;null}{true;太;フト;NandokuEngine;;0:/canna/freewnn/kakasi/mozc/;null}{true;郎;ロウ;NandokuEngine;;0:/canna/freewnn/ipadic/kakasi/mj/mozc/;null},2025-10-23 17:12:49.040 +0900,2025-10-23 17:12:49.042 +0900
4,BILLY JOEL,ビリー　ジョエル,ok,true,90,{true;ＢＩＬＬＹ;ビリー;FWordEngine;;0:/;0000000}{true;ＪＯＥＬ;ジョエル;FWordEngine;;0:/;0000000},2025-10-23 17:12:49.

# 漢字からカナを推測する
漢字姓名が入力されたファイルをCSVもしくはTSV形式で用意する。漢字姓名以外のフィールドが含まれていても良い。
なお，VMパラメタとして`-Xmx8196M`を指定し，8Gのメモリを確保すること。
### 実行クラス
jp.go.digital.kanjikana.core.executor.generate.Kanji2KanaMain
### パラメタ
- infile
入力ファイル。行に漢字姓名を含むファイルで，TSVもしくはCSV形式で作成しること。また，ヘッダがあっても良くヘッダがある場合には`has_header`パラメタを指定する。また，漢字姓名の列を`kanji_idx`パラメタで指定する
- outfile
漢字姓名から推測されたカナ姓名を出力したファイル
- n_best
カナ姓名の候補をいくつ出力するか、デフォルト5
- kanji_idx
infileのファイルの漢字姓名の列番号，0から始まる
- sep
infileの各列の区切り文字を指定する。csvもしくはtsvを指定する。デフォルトcsv
- has_header
infileにヘッダ業がある場合には`true`を指定する。デフォルトは`false`であり，infileにヘッダ行は含まれない


In [5]:
!java -Xmx8196M -Dlog4j.configurationFile=log4j2.xml -classpath lib/kanjikana_core-$VER-jar-with-dependencies.jar jp.go.digital.kanjikana.core.executor.generate.Kanji2KanaMain --infile input.txt --outfile outfile_kana.txt --kanji_idx 1  --has_header true  --sep csv


17:13:00.378 [main] DEBUG ai.djl.repository.zoo.DefaultModelZoo -- Scanning models in repo: class ai.djl.repository.SimpleRepository, /tmp/model_6612789570224081337.tmp
17:13:00.384 [main] DEBUG ai.djl.engine.Engine -- Registering EngineProvider: PyTorch
17:13:00.385 [main] DEBUG ai.djl.engine.Engine -- Found default engine: PyTorch
17:13:00.855 [main] INFO ai.djl.util.Ec2Utils -- DJL will collect telemetry to help us better understand our users’ needs, diagnose issues, and deliver additional features. If you would like to learn more or opt-out please go to: https://docs.djl.ai/docs/telemetry.html for more information.
17:13:00.858 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Loading model with Criteria:
	Application: UNDEFINED
	Input: class ai.djl.ndarray.NDList
	Output: class ai.djl.ndarray.NDList
	Engine: PyTorch
	ModelZoo: ai.djl.localmodelzoo
	No translator supplied

17:13:00.858 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Searching model in specified model zoo: ai.djl.localm

## 出力

In [6]:
!cat outfile_kana.txt

no,kanji,kana,okng,result,start_date,end_date
1,山田　太郎,ヤマダ　ハナコ,ng,kanji:山田　太郎;kanji1:山田　太郎;best:1;predict:ヤマダタロウ;probability:-2.270516720071963E-4,2025-10-23 17:12:59.371 +0900,2025-10-23 17:13:03.911 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kanji:山田　太郎;kanji1:山田　太郎;best:2;predict:ヤマダダイロウ;probability:-9.744585080123263,2025-10-23 17:12:59.371 +0900,2025-10-23 17:13:03.911 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kanji:山田　太郎;kanji1:山田　太郎;best:3;predict:ヤマダイタロウ;probability:-10.861071116937053,2025-10-23 17:12:59.371 +0900,2025-10-23 17:13:03.911 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kanji:山田　太郎;kanji1:山田　太郎;best:4;predict:ヤマダヒロオ;probability:-10.88799437673626,2025-10-23 17:12:59.371 +0900,2025-10-23 17:13:03.911 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kanji:山田　太郎;kanji1:山田　太郎;best:5;predict:ヤマダブロウ;probability:-11.247447151520818,2025-10-23 17:12:59.371 +0900,2025-10-23 17:13:03.911 +0900
2,山田　太郎,ヤマダタロウ,ok,kanji:山田　太郎;kanji1:山田　太郎;best:1;predict:ヤマダタロウ;probability:-2.270516720071963E-4,2025-10-23 17:13:03.924 +0900,2025-10-23 17:13:04.171 +0900
2,山

## 出力の見方
kanji:山田　太郎;kanji1:山田　太郎;best:1;predict:ヤマダタロウ;probability:-2.270516720071963E-4,2025-10-06 15:59:29.910 +0900,2025-10-06 15:59:34.276 +0900

- kanji: input.txtの漢字姓名
- kanji1: 旧姓など複数の名前に分けられる場合の通し番号、 山田［田中】 花子 の場合には、kanji1:山田 花子、kanji2:田中 花子になる
- best:1  nbestで指定した数だけ出力。数字が小さいほど確率が高い
- predict: 予測した漢字の読みカナ
- probability: 確率、log(probability) で確率になる。 


# カナから漢字を推測する
カナ姓名が入力されたファイルをCSVもしくはTSV形式で用意する。カナ姓名以外のフィールドが含まれていても良い。
なお，VMパラメタとして`-Xmx8196M`を指定し，8Gのメモリを確保すること。
### 実行クラス
jp.go.digital.kanjikana.core.executor.generate.Kana2KanjiMain
### パラメタ
- infile
入力ファイル。行にカナ姓名を含むファイルで，TSVもしくはCSV形式で作成しること。また，ヘッダがあっても良くヘッダがある場合には`has_header`パラメタを指定する。また，カナ姓名の列を`kana_idx`パラメタで指定する
- outfile
カナ姓名から推測された漢字姓名を出力したファイル
- n_best
漢字姓名の候補をいくつ出力するか、デフォルト5
- kana_idx
infileのファイルのカナ姓名の列番号，0から始まる
- sep
infileの各列の区切り文字を指定する。csvもしくはtsvを指定する。デフォルトcsv
- has_header
infileにヘッダ業がある場合には`true`を指定する。デフォルトは`false`であり，infileにヘッダ行は含まれない


In [7]:
!java -Xmx8196M -Dlog4j.configurationFile=log4j2.xml -classpath lib/kanjikana_core-$VER-jar-with-dependencies.jar jp.go.digital.kanjikana.core.executor.generate.Kana2KanjiMain --infile input.txt --outfile outfile_kanji.txt --kana_idx 2  --has_header true  --sep csv


17:13:13.481 [main] DEBUG ai.djl.repository.zoo.DefaultModelZoo -- Scanning models in repo: class ai.djl.repository.SimpleRepository, /tmp/model_4672223388856139893.tmp
17:13:13.486 [main] DEBUG ai.djl.engine.Engine -- Registering EngineProvider: PyTorch
17:13:13.487 [main] DEBUG ai.djl.engine.Engine -- Found default engine: PyTorch
17:13:13.949 [main] INFO ai.djl.util.Ec2Utils -- DJL will collect telemetry to help us better understand our users’ needs, diagnose issues, and deliver additional features. If you would like to learn more or opt-out please go to: https://docs.djl.ai/docs/telemetry.html for more information.
17:13:13.951 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Loading model with Criteria:
	Application: UNDEFINED
	Input: class ai.djl.ndarray.NDList
	Output: class ai.djl.ndarray.NDList
	Engine: PyTorch
	ModelZoo: ai.djl.localmodelzoo
	No translator supplied

17:13:13.951 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Searching model in specified model zoo: ai.djl.localm

In [8]:
!cat outfile_kanji.txt

no,kanji,kana,okng,result,start_date,end_date
1,山田　太郎,ヤマダ　ハナコ,ng,kana:ヤマダ　ハナコ;best:1;predict:山田花子;probability:-0.6550042959654809,2025-10-23 17:13:12.517 +0900,2025-10-23 17:13:17.226 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kana:ヤマダ　ハナコ;best:2;predict:山田華子;probability:-0.8174689959806346,2025-10-23 17:13:12.517 +0900,2025-10-23 17:13:17.226 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kana:ヤマダ　ハナコ;best:3;predict:山田はな子;probability:-4.702061071025153,2025-10-23 17:13:12.517 +0900,2025-10-23 17:13:17.226 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kana:ヤマダ　ハナコ;best:4;predict:山田はなこ;probability:-5.2323108127248315,2025-10-23 17:13:12.517 +0900,2025-10-23 17:13:17.226 +0900
1,山田　太郎,ヤマダ　ハナコ,ng,kana:ヤマダ　ハナコ;best:5;predict:山田英子;probability:-5.44509280565863,2025-10-23 17:13:12.517 +0900,2025-10-23 17:13:17.226 +0900
2,山田　太郎,ヤマダタロウ,ok,kana:ヤマダタロウ;best:1;predict:山田太郎;probability:-0.06984249788715825,2025-10-23 17:13:17.238 +0900,2025-10-23 17:13:17.445 +0900
2,山田　太郎,ヤマダタロウ,ok,kana:ヤマダタロウ;best:2;predict:山田太朗;probability:-2.924842634951389,2025-10-23 

## javadocを作成する
Javadocを作成し，/docs以下へ移動する。GithubPagesで見えるようにする。 

URLは [https://studious-chainsaw-5k2p5q6.pages.github.io/](https://studious-chainsaw-5k2p5q6.pages.github.io/)


In [9]:
!mvn javadoc:javadoc

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for jp.go.digital.kanjikana:kanjikana_core:jar:1.8o
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: net.sourceforge.argparse4j:argparse4j:jar -> duplicate declaration of version 0.9.0 @ line 58, column 21
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ---------------< jp.go.digital.kanjikana:kanjikana_core >---------------
[INFO] Building kanjikana_core 1.8o
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] >>> maven-javadoc-plugin:3.6.3:javadoc (default-cli) > generate-sources @ kanjikana_core >>>
[INFO] 
[INFO] <<< maven-javadoc-plugin:3.6.3:javadoc (default-cli) < gener

In [10]:
!date

2025年 10月 23日 木曜日 17:13:28 JST
